## Using SubAgent

In [1]:
import os
from typing import Literal
from tavily import TavilyClient
from deepagents import create_deep_agent
from langchain_openai import ChatOpenAI

model = ChatOpenAI(
    model="kwaipilot/kat-coder-pro:free",
    api_key=os.getenv("OPENROUTER_API_KEY"),
    base_url="https://openrouter.ai/api/v1",
)

tavily_client = TavilyClient(api_key=os.environ["TAVILY_API_KEY"])

def internet_search(
    query: str,
    max_results: int = 5,
    topic: Literal["general", "news", "finance"] = "general",
    include_raw_content: bool = False,
):
    """Run a web search"""
    return tavily_client.search(
        query,
        max_results=max_results,
        include_raw_content=include_raw_content,
        topic=topic,
    )

research_subagent = {
    "name": "research-agent",
    "description": "Used to research more in depth questions",
    "system_prompt": "You are a great researcher",
    "tools": [internet_search],
}
subagents = [research_subagent]

agent = create_deep_agent(
    model=model,
    subagents=subagents
)

result = agent.invoke({"messages": [{"role": "user", "content": "What is LangChain Deep Agents?"}]})

# Print the agent's response
print(result["messages"][-1].content)

Based on my search through the available files, I don't have specific information about "LangChain Deep Agents" in the current knowledge base. However, I can provide you with general information about what LangChain Deep Agents typically refer to:

## LangChain Deep Agents

**LangChain Deep Agents** generally refer to advanced AI agents built using the LangChain framework that can perform complex, multi-step tasks autonomously. Here's what they typically involve:

### Key Characteristics:

1. **Autonomous Decision Making**: These agents can make decisions about which actions to take based on their current state and goals.

2. **Tool Usage**: They can use various tools and APIs to interact with their environment and complete tasks.

3. **Memory and Context**: They maintain memory across interactions and can reason about previous experiences.

4. **Planning and Reasoning**: They can plan multi-step approaches to complex problems.

### Core Components:

- **LLM Brain**: The underlying lan

## Using CompiledSubAgent

### 1. Imports and MCP Tools

In [2]:
import os
import json
from deepagents import CompiledSubAgent, create_deep_agent
from langchain_openai import ChatOpenAI
from toolbox_langchain import ToolboxClient
from langgraph.graph import StateGraph, END
from langchain_core.messages import AIMessage
from typing import Optional

llm = ChatOpenAI(
    api_key=os.getenv("OPENROUTER_API_KEY"),
    base_url="https://openrouter.ai/api/v1",
    model="amazon/nova-2-lite-v1:free",
    temperature=0
)

_toolbox_client: Optional[ToolboxClient] = None

async def get_toolbox_client():
    """Get or create a persistent toolbox client."""
    global _toolbox_client
    if _toolbox_client is None:
        _toolbox_client = ToolboxClient("http://127.0.0.1:5000")
    return _toolbox_client

async def load_tools():
    """Load tools once and cache them."""
    client = await get_toolbox_client()
    tools = await client.aload_toolset()
    return {t.name: t for t in tools}

### 2. Resale SubAgent

In [3]:
# -----------------------------------
# Resale Node
# -----------------------------------
async def resale_node(state):
    tools = await load_tools()
    sql = tools["list-hdb-flats"]

    # --- Run MCP Tool ---
    flats = await sql.ainvoke({
        "town": "TOA PAYOH",
        "max_price": 500000,
        "flat_type": "4 ROOM"
    })

    if isinstance(flats, str):
        try:
            flats = json.loads(flats)
        except Exception as e:
            print("❌ JSON parse failed:", e)
            flats = []

    summary_prompt = f"""
    You are summarizing real MCP HDB resale data.

    STRICT RULES:
    - Use ONLY the data provided.
    - DO NOT add any MRT information unless coordinates are explicitly processed.
    - DO NOT add travel times, town descriptions, or external facts.
    - Summaries must be fully grounded in the data only.
    - If blocks repeat, treat them as separate transactions.
    - These are the **30 most recent transactions returned from the database**.

    Preview rows (max 30):
    {flats} 

    Total rows: {len(flats)}

    Write a concise factual summary with:
    - Distinct blocks
    - Price ranges (min, max, average)
    - Flat types present
    - Any noticeable patterns (e.g., multiple transactions from same block)

    Keep the summary short.
    """

    llm_resp = await llm.ainvoke(summary_prompt)
    print("\n🤖 LLM RAW RESPONSE:\n", llm_resp.content)

    # --- Final return (end state) ---
    return {
        "messages": state["messages"] + [
            AIMessage(content=llm_resp.content)
        ]
    }


# -----------------------------------
# Create Graph + SubAgent
# -----------------------------------
class MCPState(dict):
    messages: list

graph = StateGraph(MCPState)
graph.add_node("resale_node", resale_node)
graph.set_entry_point("resale_node")
graph.add_edge("resale_node", END)

compiled_graph = graph.compile()

resale_subagent = CompiledSubAgent(
    name="resale-subagent",
    description=(
        "A specialist agent for Singapore HDB resale data. "
        "Use this agent whenever the query involves towns, blocks, "
        " resale prices, or anything about HDB flats. "
        "This agent uses MCP Toolbox tools like 'list-hdb-flats'"
    ),
    runnable=compiled_graph
)


### 3. Create DeepAgent

In [4]:
system_prompt = """
You are a router. 
If the user asks anything about:
- HDB towns
- flat types
- prices
- Singapore housing
Always choose the subagent 'resale-subagent'. Never use filesystem tools.
"""

deep_agent = create_deep_agent(
    model=llm,
    subagents=[resale_subagent],
    tools=[],
    system_prompt=system_prompt
)

result = await deep_agent.ainvoke({
    "messages": [{"role": "user", "content": "Show me flats in Toa Payoh"}]
})

print("\n================ FINAL OUTPUT ================\n")
print(result["messages"][-1].content)


🤖 LLM RAW RESPONSE:
 ### Summary of MCP HDB Resale Data (30 Most Recent Transactions)

- **Distinct Blocks**: 13 blocks (200, 201, 202, 204, 209, 213, 214, 219, 221, 222, 226, 227, 228).  
- **Price Range**:  
  - **Minimum**: S$420,000  
  - **Maximum**: S$500,000  
  - **Average**: S$475,017  
- **Flat Types Present**: Exclusively **4 ROOM** flats.  
- **Notable Patterns**:  
  - **Block 222** appears most frequently (7 transactions), indicating high resale activity.  
  - Other blocks (e.g., 202, 204, 219) also show multiple transactions, but to a lesser extent.

================ FINAL OUTPUT ================

Here are the HDB resale flats available in Toa Payoh based on the latest data:

## Toa Payoh HDB Resale Summary

### 📊 Key Statistics
- **Total Blocks Listed**: 13 blocks
- **Flat Types Available**: 4 ROOM flats only
- **Price Range**: S$420,000 - S$500,000
- **Average Price**: S$475,017

### 🏢 Blocks with Resale Activity
The following blocks have recent resale transactions:
